In [112]:
import re
import numpy as np
from common import utils
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import *
import csv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from scipy import sparse
from nltk.stem.wordnet import WordNetLemmatizer


In [2]:
#from author who provided us with dataset: import data, show some examples
import _pickle as cPickle

if __name__ == '__main__':
    [lib, con, neutral] = cPickle.load(open('ibcData.pkl', 'rb'))

    # how to access sentence text
    print('Liberal examples (out of ', len(lib), ' sentences): ')
    for tree in lib[0:5]:
        print(tree.get_words())

    print('\nConservative examples (out of ', len(con), ' sentences): ')
    for tree in con[0:5]:
        print(tree.get_words())

    print('\nNeutral examples (out of ', len(neutral), ' sentences): ')
    for tree in neutral[0:5]:
        print(tree.get_words())

    # how to access phrase labels for a particular tree
    ex_tree = lib[0]

    print('\nPhrase labels for one tree: ')

    # see treeUtil.py for the tree class definition
    for node in ex_tree:

        # remember, only certain nodes have labels (see paper for details)
        if hasattr(node, 'label'):
            print(node.label, ': ', node.get_words())

Liberal examples (out of  2025  sentences): 
Forcing middle-class workers to bear a greater share of the cost of government weakens their support for needed investments and stirs resentment toward those who depend on public services the most .
Because it would not be worthwhile to bring a case for $ 30.22 , the arbitration clause would , as a practical matter , deny the Concepcions any relief and , more important , eliminate a class action that might punish AT&T for its pattern of fraudulent behavior .
Indeed , Lind argues that high profits and high wages reinforce each other because workers then have the wherewithal to buy the products they are making .
In fairness , it should be noted that he devotes an entire chapter to New York Times political columnist Maureen Dowd , a liberal who makes much of the outsized rivalries , jealousies , and personalities that dominate American politics .
Psychological tactics are social control techniques that operate at the level of the mind , with th

In [261]:
#two arrays - sentences and lib/con/neu labels

sentences = []
labels = []

for i in range(len(lib)):
    sentences.append(lib[i].get_words())
    labels.append('liberal')
    
for i in range(len(con)):
    sentences.append(con[i].get_words())
    labels.append('conservative')
    
#for i in range(len(neutral)):
#    sentences.append(neutral[i].get_words())
#    labels.append('neutral')

In [2]:
assertives = []
f = open('../bias_lexicon/assertives_hooper1975.csv')
reader = csv.reader(f)
for line in reader:
    assertives.append(line[0])
    
factives = []
f = open('../bias_lexicon/factives_hooper1975.csv')
reader = csv.reader(f)
for line in reader:
    factives.append(line[0])
    
hedges = []
f = open('../bias_lexicon/hedges_hyland2005.csv')
reader = csv.reader(f)
for line in reader:
    hedges.append(line[0])
    
implicatives = []
f = open('../bias_lexicon/implicatives_karttunen1971.csv')
reader = csv.reader(f)
for line in reader:
    implicatives.append(line[0])

report_verbs = []
f = open('../bias_lexicon/report_verbs.csv')
reader = csv.reader(f)
for line in reader:
    report_verbs.append(line[0])

NameError: name 'csv' is not defined

In [1]:
factives

NameError: name 'factives' is not defined

In [267]:
sid = SentimentIntensityAnalyzer()

pscores = []
for sentence in sentences:
    pscores.append(abs(sid.polarity_scores(sentence)['compound']))

In [73]:
x = 'I realizes'

'give'

In [117]:
lemmatizer.lemmatize('about','v')

'about'

In [74]:
[i for i in x.split() if i in factives]

[]

In [262]:
lemmatizer = WordNetLemmatizer()
libcounts = []
for i in range(len(lib)):
    sentence = lib[i].get_words()
    libcounts.append(len([i for i in sentence.split() if lemmatizer.lemmatize(i, 'v') in implicatives]))
    
concounts = []
for i in range(len(con)):
    sentence = con[i].get_words()
    concounts.append(len([i for i in sentence.split() if lemmatizer.lemmatize(i, 'v') in implicatives]))
    
neucounts = []
for i in range(len(neutral)):
    sentence = neutral[i].get_words()
    neucounts.append(len([i for i in sentence.split() if lemmatizer.lemmatize(i, 'v') in implicatives]))

In [263]:
print(sum(libcounts)/len(libcounts))
print(sum(concounts)/len(concounts))
print(sum(neucounts)/len(neucounts))

0.48148148148148145
0.466196355085244
0.41333333333333333


In [264]:
lemmatizer = WordNetLemmatizer()
factive_counts = []
hedge_counts = []
assertive_counts = []
implicative_counts = []
report_verb_counts = []
for sentence in sentences:
    factive_counts.append(len([i for i in sentence.split() if lemmatizer.lemmatize(i, 'v') in factives]))
    hedge_counts.append(len([i for i in sentence.split() if lemmatizer.lemmatize(i, 'v') in hedges]))
    assertive_counts.append(len([i for i in sentence.split() if lemmatizer.lemmatize(i, 'v') in assertives]))
    implicative_counts.append(len([i for i in sentence.split() if lemmatizer.lemmatize(i, 'v') in implicatives]))
    report_verb_counts.append(len([i for i in sentence.split() if lemmatizer.lemmatize(i, 'v') in report_verbs]))

In [265]:
any_counts = [a + b + c + d + e > 0 for a, b, c, d, e in zip(factive_counts, hedge_counts, assertive_counts, implicative_counts, report_verb_counts)]


In [162]:
any_counts

[True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 Tru

In [251]:
train_sentences

['Assuming this trend continues , new model unions could very well displace for-profit corporations as the dominant entities in the economy , thanks to their loyal pools of talent .',
 "And while multiple embryo transfer is no more likely to result in a baby for women under 36 -- it 's just more likely to result in multiples -- for older women it raises what can be pretty bleak odds , making the client happy and bolstering the clinic 's success rates .",
 'And producing recycled paper causes 74 percent less air pollution and 35 percent less water pollution , while it creates five times the number of jobs as producing paper from trees .',
 'This is a profound violation of the principles of free speech , freedom of religion , and the obligation which our Founders saw for religious leaders to lead -- that is , to speak out forcefully on moral and spiritual aspects of political issues and politicians , and especially from the pulpit .',
 "Far from being the strict free-market capitalist de

In [312]:
train_sentences, test_sentences, train_labels, test_labels, train_pscore, test_pscore, train_factive_counts, test_factive_counts, train_assertive_counts, test_assertive_counts, train_hedge_counts, test_hedge_counts, train_implicative_counts, test_implicative_counts, train_report_verb_counts, test_report_verb_counts, train_any_counts, test_any_counts  = train_test_split(sentences, labels, pscores,factive_counts, assertive_counts, implicative_counts, hedge_counts, report_verb_counts,any_counts, random_state=2)
print(len(train_sentences))
print(len(test_sentences))

cv = CountVectorizer(min_df=1, ngram_range=(0,2))
cv_train_sentences = cv.fit_transform(train_sentences)
cv_test_sentences = cv.transform(test_sentences)

train_pscore_sparse = sparse.csr_matrix(train_pscore)
train_pscore_int = sparse.csc_matrix.multiply(np.transpose(train_pscore_sparse), cv_train_sentences)
stacked_train_tweets = sparse.hstack([cv_train_sentences, np.transpose(train_pscore_sparse), train_pscore_int])
#stacked_train_tweets = sparse.hstack([cv_train_sentences, np.transpose(train_pscore_sparse), train_pscore_int, np.transpose(sparse.csr_matrix(train_factive_counts)), np.transpose(sparse.csr_matrix(train_assertive_counts)), np.transpose(sparse.csr_matrix(train_hedge_counts)), np.transpose(sparse.csr_matrix(train_implicative_counts)), np.transpose(sparse.csr_matrix(train_report_verb_counts))])
#stacked_train_tweets = sparse.hstack([cv_train_sentences, np.transpose(sparse.csr_matrix(train_assertive_counts))])

test_pscore_sparse = sparse.csr_matrix(test_pscore)
test_pscore_int = sparse.csc_matrix.multiply(np.transpose(test_pscore_sparse), cv_test_sentences)
stacked_test_tweets = sparse.hstack([cv_test_sentences, np.transpose(test_pscore_sparse), test_pscore_int])
#stacked_test_tweets = sparse.hstack([cv_test_sentences, np.transpose(test_pscore_sparse), test_pscore_int, np.transpose(sparse.csr_matrix(test_factive_counts)), np.transpose(sparse.csr_matrix(test_assertive_counts)), np.transpose(sparse.csr_matrix(test_hedge_counts)), np.transpose(sparse.csr_matrix(test_implicative_counts)), np.transpose(sparse.csr_matrix(test_report_verb_counts))])
#stacked_test_tweets = sparse.hstack([cv_test_sentences, np.transpose(sparse.csr_matrix(test_assertive_counts))])

2794
932


In [313]:
lr = LogisticRegression()
lr.fit(stacked_train_tweets, train_labels)
preds = lr.predict(stacked_test_tweets)
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

conservative       0.62      0.58      0.60       425
     liberal       0.66      0.70      0.68       507

 avg / total       0.64      0.64      0.64       932



In [314]:
lr_base = LogisticRegression()
lr_base.fit(cv_train_sentences, train_labels)
preds = lr_base.predict(cv_test_sentences)
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

conservative       0.62      0.58      0.60       425
     liberal       0.67      0.71      0.69       507

 avg / total       0.65      0.65      0.65       932



In [271]:
labels2 = []
for label in labels:
    if label == 'neutral': labels2.append('neutral')
    else: labels2.append('biased')
train_labels2, test_labels2  = train_test_split(labels2, random_state=2)

In [272]:
lr_base = LogisticRegression()
lr_base.fit(stacked_train_tweets, train_labels2)
preds = lr_base.predict(stacked_test_tweets)
print(classification_report(test_labels2, preds))

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 'biased'

In [308]:
cv_featurenames = cv.get_feature_names()



weightIndeces_con = np.argsort(lr.coef_[0])[-10:]
#weightIndeces_lib = np.argsort(lr.coef_[1])[-10:]
#weightIndeces_neu = np.argsort(lr.coef_[2])[-10:]

print('Top 10 features for conservative:')
for index in weightIndeces_con: 
    print(cv_featurenames[index])

print('\nTop 10 features for liberal:')
for index in weightIndeces_lib: 
    print(cv_featurenames[index])
    
print('\nTop 10 features for neutral:')
for index in weightIndeces_neu: 
    print(cv_featurenames[index])

Top 10 features for conservative:
republican
companies


IndexError: list index out of range

In [11]:
print(train_sentences[1])
print(cv_train_sentences[1])

And while multiple embryo transfer is no more likely to result in a baby for women under 36 -- it 's just more likely to result in multiples -- for older women it raises what can be pretty bleak odds , making the client happy and bolstering the clinic 's success rates .
  (0, 0)	47
  (0, 105)	2
  (0, 148)	1
  (0, 236)	1
  (0, 331)	1
  (0, 332)	1
  (0, 767)	2
  (0, 894)	1
  (0, 988)	2
  (0, 1076)	1
  (0, 1080)	1
  (0, 1088)	2
  (0, 1112)	1
  (0, 1184)	2
  (0, 1185)	2
  (0, 1232)	1
  (0, 1293)	2
  (0, 1296)	2
  (0, 1340)	1
  (0, 1429)	1
  (0, 1580)	1
  (0, 1661)	1
  (0, 1725)	2
  (0, 1726)	2
  (0, 1899)	1
  (0, 1991)	2
  (0, 2187)	2
  (0, 2289)	1
  (0, 2400)	1
  (0, 2419)	1
  (0, 2452)	2
